In [3]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import IPython as ip
import  os as os

In [4]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [ ]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file

In [79]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

In [80]:
##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

,borrows_id,borrows_hash,borrows_logIndex,borrows_blockNumber,borrows_timestamp,borrows_account_id,borrows_account_positionCount,borrows_account_openPositionCount,borrows_account_closedPositionCount,borrows_account_depositCount,...,borrows_asset_type,borrows_asset_lastPriceUSD,borrows_asset_lastPriceBlockNumber,borrows_asset_market_id,borrows_asset_asset_id,borrows_asset_derivativeConversion,borrows_asset_totalSupply,borrows_asset_activeOracle_id,borrows_amount,borrows_amountUSD
0,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...,186000000000000000000000,185996.147731
1,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...,186000000000000000000000,185996.147731
2,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...,186000000000000000000000,185996.147731
3,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...,186000000000000000000000,185996.147731
4,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...,186000000000000000000000,185996.147731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...,182000000000000000,399.040935
92,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...,182000000000000000,399.040935
93,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...,182000000000000000,399.040935
94,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...,182000000000000000,399.040935


In [81]:
##this captures the list of people who borrowed XAI against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
borrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.logIndex
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account
        ,borrows.market
        ,borrows.market.rates
        ,borrows.asset
        ,borrows.amount
        ,borrows.amountUSD
        ,borrows.position
    ])

    #borrowslist.sort_values(by=['borrows_timestamp'])

    borrowsDb=pd.concat([borrowsDb, borrowslist])
    recordTimestamp1 = borrowsDb.iat[borrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    borrowsDbLength = len(borrowsDb)
    borrowListLength = len(borrowslist)
    recordID = borrowsDb.iat[borrowTicker,0]
    borrow = borrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", borrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

borrowsDb.columns = borrowsDb.columns.str.replace("borrows", "transaction")
borrowsDb['event']='borrow'
print("Done. Total Records collected: ", borrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  96  - latest recordID:  0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657   0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657-105  - borrows DB length:  96  - latest record from:  2023-02-08 22:23:59 1 days ago
Done. Total Records collected:  96


In [82]:
borrowsDb.head(20)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
1,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
2,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
3,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
4,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
5,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
6,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
7,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
8,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
9,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow


In [83]:
#this tracks the events for depositing gOHM for lending
##this captures the list of people who deposited XAI against gOHM
depositTicker = 0
depositslist = pd.DataFrame()
depositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account
        ,deposits.market
        ,deposits.market.rates
        ,deposits.asset
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits.position
    ])

    #depositslist.sort_values(by=['deposits_timestamp'])

    depositsDb=pd.concat([depositsDb, depositslist])
    recordTimestamp1 = depositsDb.iat[depositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    depositsDbLength = len(depositsDb)
    depositListLength = len(depositslist)
    recordID = depositsDb.iat[depositTicker,0]
    deposit = depositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", depositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", depositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1
    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

#depositsDb.sort_values(by=['deposits_timestamp'])

depositsDb.columns = depositsDb.columns.str.replace("deposits", "transaction")
depositsDb['event']='deposit'
print("Done. Total Records collected: ", depositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  222  - latest recordID:  0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81   0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81-204  - deposits DB length:  222  - latest record from:  2023-02-09 09:48:11 0 days ago
Done. Total Records collected:  222


In [84]:
depositsDb.head(20)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
5,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
6,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
7,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit


In [85]:
siloEventsDb = pd.DataFrame()
siloEventsDb=pd.concat([depositsDb, borrowsDb])
siloEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
5,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
6,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
7,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit


In [86]:
#I swear I will figure out synthetic fields, but until then...
siloEventsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From siloEventsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
siloEventsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0,1,1,0,0,0,deposit,2023-01-09 02:05:11
96,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35
97,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35
98,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35


In [87]:
siloEventsDblength = len(siloEventsDbTimed)
siloEventsDblength

318

In [88]:
path =file+'/siloFinance_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
siloEventsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
siloEventsDbTimed.to_csv(path, index = False, sep='|')

In [89]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = silo.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
        ,interestRates.market
        ,interestRates.marketAsset
        ,interestRates.token
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  6
Done. Total Records collected:  6


In [90]:
interestRatesDb.head(10)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type,interestRates_market_id,interestRates_market_protocol_id,interestRates_market_name,interestRates_market_isActive,...,interestRates_token_symbol,interestRates_token_decimals,interestRates_token_type,interestRates_token_lastPriceUSD,interestRates_token_lastPriceBlockNumber,interestRates_token_market_id,interestRates_token_asset_id,interestRates_token_derivativeConversion,interestRates_token_totalSupply,interestRates_token_activeOracle_id
0,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,2.148783,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
1,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1538.963318,16595930,None,None,1.0,0,None
2,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...
3,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,3.277791,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
4,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1538.963318,16595930,None,None,1.0,0,None
5,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...


In [91]:
path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')

LET'S LOOK AT VESTA WHILE WE'RE ON A RUN

In [150]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance')

In [131]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    vestaBorrows = vesta.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.hash
        ,vestaBorrows.logIndex
        ,vestaBorrows.blockNumber
        ,vestaBorrows.timestamp
        ,vestaBorrows.account
        ,vestaBorrows.market
        ,vestaBorrows.market.rates
        ,vestaBorrows.asset
        ,vestaBorrows.amount
        ,vestaBorrows.amountUSD
        ,vestaBorrows.position
    ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d   borrow-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d-38  - vestaBorrows DB length:  1000  - latest record from:  2023-02-09 12:33:03 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  397  - latest recordID:  0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee   borrow-0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee-37  - vestaBorrows DB length:  1397  - latest record from:  2023-02-08 20:04:50 1 days ago
Done. Total Records collected:  1397


In [132]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(borrows_timestamp::int) dateTime "
               "From vestaBorrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,borrow-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,BORROWER,NaN,550000374479268793127538,0,0,4,150,0,borrow,2023-02-09 20:33:03
1,borrow-0x5d3025a16815324cf1afca689903a14d2621c...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,BORROWER,NaN,600000064003338503382,0,0,3,0,0,borrow,2023-02-09 04:04:50
2,borrow-0x9285593b5e82d62c3211c6ea2b1ae37878091...,0x9285593b5e82d62c3211c6ea2b1ae378780919897063...,4,59127942,1675866088,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,BORROWER,NaN,0,0,0,1,1,0,borrow,2023-02-08 14:21:28
3,borrow-0x6150d3df67aa2f3cb0c5041d516a4e0f3cffc...,0x6150d3df67aa2f3cb0c5041d516a4e0f3cffccfe70d4...,36,58934904,1675812656,0x394b67c6bc05abb14c73a57706dcd5cb85231c4e,2,2,0,1,...,BORROWER,NaN,1669620110651309523172,0,0,3,1,0,borrow,2023-02-07 23:30:56
4,borrow-0x66b8e892882e3f5d0a6bd0aa81ba9d5cec32e...,0x66b8e892882e3f5d0a6bd0aa81ba9d5cec32ec4cd786...,26,58785787,1675771382,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,BORROWER,NaN,78903329999999999121255,0,0,6,1,0,borrow,2023-02-07 12:03:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,borrow-0x39e62f0bf2b26fb415c18eed751f3c6148f27...,0x39e62f0bf2b26fb415c18eed751f3c6148f271f70ca5...,23,47668312,1671580995,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,BORROWER,NaN,78903329999999999121255,0,0,6,1,0,borrow,2022-12-21 00:03:15
96,borrow-0x965180bee5e4d28e5299cc0c996778fa816f3...,0x965180bee5e4d28e5299cc0c996778fa816f3b9646ec...,34,47474556,1671516834,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7,2,5,5,...,BORROWER,NaN,105000101450474620590108,0,0,11,0,0,borrow,2022-12-20 06:13:54
97,borrow-0xcd8fdb79f9c0f0ebda33b2ceb25fbc541a246...,0xcd8fdb79f9c0f0ebda33b2ceb25fbc541a24624636e3...,34,47370369,1671471134,0x53430bde5e57ed878406a20ca101b0853f173acf,4,2,2,2,...,BORROWER,NaN,3614750313057953658431,0,0,4,2,0,borrow,2022-12-19 17:32:14
98,borrow-0xb99ce9341425e4d0c4b48669847eebbe6a4f1...,0xb99ce9341425e4d0c4b48669847eebbe6a4f1cbfad75...,34,47344538,1671462599,0xf352e5320291298be60d00a015b27d3960f879fa,5,1,4,4,...,BORROWER,NaN,0,0,0,1,1,0,borrow,2022-12-19 15:09:59


In [133]:
path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')

In [146]:
#this tracks the events for vestaDepositing gOHM for lending 
vestaDepositTicker = 0
vestaDepositslist = pd.DataFrame()
vestaDepositsDb = pd.DataFrame()
vestaDepositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDepositTicker)*(1000)
    vestaDeposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    vestaDepositslist = sg.query_df([
        vestaDeposits.id
        ,vestaDeposits.hash
        ,vestaDeposits.logIndex
        ,vestaDeposits.blockNumber
        ,vestaDeposits.timestamp
        ,vestaDeposits.account
        ,vestaDeposits.market#.id
        ,vestaDeposits.market
        ,vestaDeposits.market.rates
        #,vestaDeposits.market.rates.id #PAY NO NEVERMIND
        #,vestaDeposits.market.rates.rate
        #,vestaDeposits.market.rates.duration
        #,vestaDeposits.market.rates.maturityBlock
        #,vestaDeposits.market.rates.side
        #,vestaDeposits.market.rates.type
        ,vestaDeposits.asset
        ,vestaDeposits.amount
        ,vestaDeposits.amountUSD
        ,vestaDeposits.position
    ])

    #vestaDepositslist.sort_values(by=['vestaDeposits_timestamp'])

    vestaDepositsDb=pd.concat([vestaDepositsDb, vestaDepositslist])
    recordTimestamp1 = vestaDepositsDb.iat[vestaDepositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDepositsDbLength = len(vestaDepositsDb)
    vestaDepositListLength = len(vestaDepositslist)
    recordID = vestaDepositsDb.iat[vestaDepositTicker,0]
    vestaDeposit = vestaDepositsDb.iat[vestaDepositTicker,1]
    print("iterations: ", vestaDepositTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDepositListLength, " - latest recordID: ",vestaDeposit, " ", recordID , " - vestaDeposits DB length: ", vestaDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDepositTicker, "records collected: ", vestaDepositListLength, " - userdb length: ", vestaDepositsDbLength)
    vestaDepositTicker = vestaDepositTicker+1
    if daysAgo>daysLimit: exit =True
    if vestaDepositListLength<1000: exit =True

#vestaDepositsDb.sort_values(by=['vestaDeposits_timestamp'])

#vestaDepositsDb.columns = vestaDepositsDb.columns.str.replace("deposits", "transaction")
vestaDepositsDb['event']='deposit'
print("Done. Total Records collected: ", vestaDepositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  222  - latest recordID:  0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81   0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81-204  - vestaDeposits DB length:  222  - latest record from:  2023-02-09 09:48:11 0 days ago
Done. Total Records collected:  222


In [147]:
vestaDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
5,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
6,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
7,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit


In [148]:
vestaDepositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaDepositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaDepositsDb.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,1.673230e+09,LENDER,True,0,1,1,0,0,0,deposit
96,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,NaN,LENDER,True,20000000000000000,1,0,0,0,0,deposit
97,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,NaN,LENDER,True,20000000000000000,1,0,0,0,0,deposit
98,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,NaN,LENDER,True,20000000000000000,1,0,0,0,0,deposit


In [149]:
path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')

##this WILL Capture the list of people who lended gOHM for others to borrow
lendTicker = 0
lendslist = pd.DataFrame()
lendsDb = pd.DataFrame()
lendListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (lendTicker)*(1000)
    lends = silo.Query.lends(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    lendslist = sg.query_df([
        lends.id
        ,lends.hash
        ,lends.logIndex
        ,lends.blockNumber
        ,lends.timestamp
        ,lends.account
        ,lends.market
        ,lends.asset
        ,lends.amount
        ,lends.amountUSD
    ])

    #lendslist.sort_values(by=['lends_timestamp'])

    lendsDb=pd.concat([lendsDb, lendslist])
    recordTimestamp1 = lendsDb.iat[lendTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    lendsDbLength = len(lendsDb)
    lendListLength = len(lendslist)
    recordID = lendsDb.iat[lendTicker,0]
    lend = lendsDb.iat[lendTicker,1]
    print("iterations: ", lendTicker, "Lines skipped: ",skipValue, "records collected: ", lendListLength, " - latest recordID: ",lend, " ", recordID , " - lends DB length: ", lendsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", lendTicker, "records collected: ", lendListLength, " - userdb length: ", lendsDbLength)
    lendTicker = lendTicker+1
    if daysAgo>daysLimit: exit =True
    if lendListLength<1000 exit =True

#lendsDb.sort_values(by=['lends_timestamp'])
print("Done. Total Records collected: ", lendsDbLength)